In [40]:
using MAT, Statistics, LinearAlgebra,Random, Plots, NonlinearSolve
using StaticArrays, ForwardDiff, Distributed, SharedArrays, SciMLNLSolve, SimpleNonlinearSolve
include("mvnrnd.jl")

mvnrnd (generic function with 1 method)

In [2]:
function string_as_varname(s::AbstractString,v::Any)
    s=Symbol(s)
    if size(v,2) == 1 # check if v is a column vector
        @eval (($s) = (vec($v)))
    else
        @eval (($s) = ($v))
    end
end

file = matopen("simulationData.mat")

#Reads in ju[IO, price, Omega, data, labor, mu, aggtfp, alpha, grossy, stfp, beta, agggdp, Sigma, capital, vadd]

varnames = names(file)
for varname in varnames
    val = read(file, varname)
    # Instantiate the variable with its name in the current scope
    string_as_varname(varname,val)
    end

close(file)



In [3]:
function getVariables(year)
    IO = data[data[:,1] .== year,:]
    temp = [8,60,62,80:88...]
    IO = IO[setdiff(1:end,temp),setdiff(1:end,[1,2,3,4,5,94])]
    IO = IO[:,setdiff(1:end,temp)]
    Ω = IO ./ sum(IO,dims=2)
    α = vadd[:,year-1959] ./ grossy[:,year-1959];
    β = (grossy[:,year-1959]' * (I-diagm(1 .- α)*Ω))';
    β[beta .< 0] .= 0
    β = β / sum(β)
    λ = (inv(I - diagm(1 .- alpha)*Ω)' * β);
    L = λ .* α
    
    return α,β,Ω,L,λ

end

getVariables (generic function with 1 method)

In [4]:
domar_weights = (inv(I - diagm(1 .- alpha) * Omega)' * beta);
L = domar_weights .* alpha;

In [5]:
ε = .5;
θ = 0.001;
σ = .9;

In [1]:
function problem(X, A, β, Ω, α, ε, θ, σ,L)
    N = length(α)
    p = @view X[1:N]
    y = @view X[N+1:end]
    
    Out = zeros(2*N)
    
    q = (Ω * p .^ (1-θ)) .^ (1 / (1 - θ))
    w = p .* (A .^ ((ε - 1)/ε)) .* (α .^ (1 / ε)) .* (y .^ (1/ε)) .* L .^ (-1/ε)
    C = w' * L
  
    Out[1:N] = p - (A .^ (ε - 1) .* (α .* w .^ (1- ε) + (1 .- α) .* q .^ (1 - ε))) .^ (1/(1-ε))
    Out[N+1:end] = y - p .^ (-θ) .* (Ω'  * (p .^ ε .* A .^ (ε-1) .* q .^(θ-ε) .* (1 .- α) .* y)) - C * p .^(-σ) .* β
    
    return Out
end

problem (generic function with 1 method)

In [53]:
function problemJacobian(X, A, β, Ω, α, ε, θ, σ,L)
    N = length(α)
    p = @view X[1:N]
    y = @view X[N+1:end]
    
    Out = zeros(eltype(X),2*N)
    
    q = (Ω * p .^ (1-θ)) .^ (1 / (1 - θ))
    w = p .* (A .^ ((ε - 1)/ε)) .* (α .^ (1 / ε)) .* (y .^ (1/ε)) .* L .^ (-1/ε)
    C = w' * L
  
    
    DQDP = (q .^ θ * (p.^(-θ))') .* Ω;
    DWDP = diagm(A.^((ε-1)/ε) .* α.^(1/ε) .* y.^(1/ε) .* (1 ./ L).^(1/ε));
    DWDY = (1/ε) * diagm(p .* A.^((ε-1)/ε) .* α.^(1/ε).* (y.^(1/ε-1)) .* L.^(-1/ε));
    DCDP = DWDP' * L; 
    DCDY = DWDY' * L;

    DOut1DP = I - diagm(diagm(A)^(-1) * ((α .* (w .^ (1-ε)) + (1 .- α) .* (q.^(1-ε)))).^(ε/(1-ε))) * (diagm(α) * diagm(w.^(-ε))* DWDP + diagm(1 .-α)*diagm(q.^(-ε))*DQDP);

    DOut1DY =  -diagm(diagm(A.^(-1))*((α.*(w.^(1-ε))+(1 .- α).*(q.^(1-ε)))).^(ε/(1-ε)))*(diagm(α)*diagm(w.^(-ε))*DWDY);

    DOut2DP = -(ε * diagm(p.^(-θ))*Ω'*diagm((p.^(ε-1)).*(y).*(q.^(θ-ε)).*(1 .- α).*(A.^(ε-1)))
        + (θ-ε)*diagm(p.^(-θ))*Ω'*diagm((p.^(ε)).*(y).*(q.^(θ-ε-1)).*(1 .- α).*(A.^(ε-1)))*DQDP 
        - σ*diagm(β.*p.^(-σ-1))*C + broadcast(*, β.*(p.^(-σ)), DCDP')
        - θ* diagm(p.^(-θ-1)).*diagm(Ω'*diagm((p.^(ε)).*(q.^(θ-ε)).*(1 .- α).*(A.^(ε-1)))*y));

    DOut2DY = I - (diagm(p)^ε*diagm(A)^(ε-1)*diagm(q)^(θ-ε)*diagm(1 .- α)*Ω*diagm(p)^(-θ))' - broadcast(*, β.*(p.^(-σ)), DCDY');
    OutDeriv = [DOut1DP DOut1DY; DOut2DP DOut2DY];

end

problemJacobian (generic function with 1 method)

In [12]:
trials = 10;
GDP = zeros(trials)
λ = zeros(76,trials)


f = NonlinearFunction((u,p) -> problem(u,p,beta,Omega,alpha,ε,θ,σ,L), jac = (u,p) -> Jacobian(u,p,beta,Omega,alpha,ε,θ,σ,L))

for k in 1:trials
    
        
    A = exp.(mvnrnd(-1/2*diag(Sigma),diagm(diag(Sigma))));
    init = Complex.([exp.(-inv(I - diagm(1 .- alpha)*Omega)*log.(A))..., domar_weights ./exp.(-inv(I - diagm(1 .- alpha) * Omega)*log.(A))...]);
    
    
    p = A
    ProbN = NonlinearProblem(f,init,p)

    x = real.(solve(ProbN,NewtonRaphson(), reltol = 1e-14,abstol = 1e-14).u)
    
    
    #println(maximum(abs.(problem(x2,A,beta,Omega,alpha,ε,θ,σ,L))))
    GDP[k] = (x[1:76] .* (A .^ ((ε-1)/ε)).*(alpha.^(1/ε)).*(x[77:152].^(1/ε)).* L .^ (-1/ε))' * L;
    #λ[:,k] = (x[1:76] .* x[77:end]) / GDP[k] 
        
end

In [13]:
GDP

10-element Vector{Float64}:
   0.9859677942788871
   1.0162335559851077
 NaN
   1.0002382053471717
   1.0000392093067307
   0.9817155631233918
   1.0001221345948743
   0.9823564117418633
   1.0096125135291887
   1.0042284094429574

In [54]:
year = 1999
M = 10

a = 0.7;
b = 1.3;



(α,β,Ω,L,λ) = getVariables(1982);
f = NonlinearFunction((u,p) -> problem(u,p,β,Ω,α,ε,θ,σ,L), Jacobian = (u,p) -> problemJacobian(u,p,β,Ω,α,ε,θ,σ,L))



GDP = zeros(20,3)
sectors = [7 8 53]
sectorShocks = [LinRange(1,a,M)..., LinRange(1,b,M)...]

for i in 1:3
    
    x0 = Complex.([ones(76,1)...,λ...]);
    
    for k in 1:M
        A  = ones(76)
        A[sectors[i]] = sectorShocks[k]
        
        
    
    
        p = A
        ProbN = NonlinearProblem(f,x0,p)
        
        sol = (solve(ProbN,SimpleNewtonRaphson(autodiff = false), reltol = 1e-8,abstol = 1e-10))
        
        x = sol.u
        
        p = real.(x[1:76]) 
        y = real.(x[77:152])
        GDP[k,i] = (p .* (A .^ ((ε-1)/ε)).*(alpha.^(1/ε)).*(y.^(1/ε)).* L .^ (-1/ε))' * L
        progress = (i*k) * (10/ 6)
        
        println("$progress %")
        x0 =  x
    end
    
    x0 = Complex.([ones(76,1)...,λ...]);

    
    for k in 1:M
        A  = ones(76)
        A[sectors[i]] = sectorShocks[k + M]
        
        
    
    
        p = A
        ProbN = NonlinearProblem(f,x0,p)
        
        x = (solve(ProbN,SimpleTrustRegion(autodiff = false), reltol = 1e-8,abstol = 1e-10).u)

        

        p = real.(x[1:76]) 
        y = real.(x[77:152])
        
        GDP[k + M,i] = (p .* (A .^ ((ε-1)/ε)).*(alpha.^(1/ε)).*(y.^(1/ε)).* L .^ (-1/ε))' * L
        progress = (i*k) * (10/ 6)
        x0 =  x
    end

end


LoadError: MethodError: no method matching (NonlinearFunction{false, SciMLBase.FullSpecialize})(::var"#109#111"; Jacobian=var"#110#112"())
[0mClosest candidates are:
[0m  (NonlinearFunction{iip, specialize})(::Any; mass_matrix, analytic, tgrad, jac, jvp, vjp, jac_prototype, sparsity, Wfact, Wfact_t, paramjac, syms, paramsyms, observed, colorvec, sys) where {iip, specialize} at ~/.julia/packages/SciMLBase/VdcHg/src/scimlfunctions.jl:3478[91m got unsupported keyword argument "Jacobian"[39m

In [227]:

#f = NonlinearFunction(f!, jac = j!)

f = NonlinearFunction((u,p) -> problem(u, p...), jac = (u,p) -> real.(problemJacobian(u,p...)))
A  = ones(76)
A[7] = 0.8

x0 = x.u

p = [A,β,Ω,α,ε,θ,σ,L]


ProbN = NonlinearProblem(f,x0,p)
x = (solve(ProbN,NewtonRaphson(), reltol = 1e-16,abstol = 1e-16))

#dump(x)

#display(x.u)

maximum(abs.(problem(x.u,A,β,Ω,α,ε,θ,σ,L)))

8.881784197001252e-16

In [231]:
x = problem(x,p...)

152-element Vector{ComplexF64}:
  -8.534303539499612e-17 - 3.641045353332143e-20im
  -9.743371238137484e-19 - 4.5463571317819217e-20im
   2.228833754214963e-16 - 2.533811889656108e-20im
  -5.299412668143194e-17 - 1.6820275068467037e-20im
 -2.2530718831082355e-16 - 3.4475458580636753e-20im
 -1.0777975383050396e-16 - 2.6654791212159166e-20im
  1.1196225528492027e-15 - 5.546131412917758e-20im
  -9.398150968724914e-17 - 5.920296191229571e-20im
 -1.0901586904036594e-16 - 9.590296190210593e-20im
  1.9791571433733308e-17 - 5.703034434245937e-20im
   2.039929386620505e-16 - 5.028937949882226e-20im
  1.6085815377775795e-16 - 5.3129142786604283e-20im
 -2.4543075132795868e-16 - 6.16218145801843e-20im
                         ⋮
    6.87949312762392e-19 - 7.989528096919427e-20im
  -2.666675945011684e-19 + 4.173866569285964e-19im
                     NaN + NaN*im
   6.938893903907228e-18 + 0.0im
   8.673617379884035e-19 + 0.0im
   5.585417763647086e-36 + 1.8148122430980895e-36im
 -1.3803086493228677

In [166]:
extrema(real.(problemJacobian(sols,A,β,Ω,α,ε,θ,σ,L)))

(-338.6902024899553, 0.997138358643352)